In [1]:
def p_gu(p1, p2, n1, n2):
    return (p1*n1+p2*n2)/(n1+n2)
def z(p1,p2,n1,n2):
    return (p1-p2)/np.sqrt(p_gu(p1, p2, n1, n2)*(1-p_gu(p1, p2, n1, n2))*(1/n1 + 1/n2))
######################################################
### 函数功能：设置 pd 的部分特征为特定的数据类型   ###
### 参数解释：                                     ###
###     df:      需要设置数据类型的 pandas         ###
###     col:     需要设置特定类型的 pd 的 columns  ###
###     types:   需要设置的特定数据类型            ###
def set_type(df,col,types):
    for each in col: 
        df[col] = df[col].astype(types)
    return df
########################################################
### 函数功能：count 一个 list 中 特定 str 有多少个   ###
### 参数解释：                                       ###
###     lists:  在该 list 中进行 count               ###
###     strs:   需要 count 的 str                    ###
def count_str(lists, strs):
    count=0
    for i in range(len(lists)):
        if strs == lists[i]:
            count+=1
    return count
########################################################
### 函数功能：专门处理 payment_state 变量，统计每个  ###
###           元素中各个情况（/、*、N、1、#）分别有  ###
###           多少                                   ###
### 参数解释：                                       ###
###     df: 就是cd_ln                                ###
def count_payment_state(df,strs):
    df = df[['report_id','payment_state']].drop(df[df.payment_state.isnull()].index,axis=0)
    count_payment_state_A = []  # /
    count_payment_state_B = []  # *
    count_payment_state_C = []  # N
    count_payment_state_D = []  # 1
    count_payment_state_E = []  # #
    
    for each_row in range(len(df)):
        num_斜杠 = count_str(df.iloc[each_row,1], '/') 
        num_星号 = count_str(df.iloc[each_row,1], '*') 
        num_N = count_str(df.iloc[each_row,1], 'N')
        num_1 = count_str(df.iloc[each_row,1], '1')
        num_3 = count_str(df.iloc[each_row,1], '#')
        
        count_payment_state_A.append(num_斜杠)
        count_payment_state_B.append(num_星号)
        count_payment_state_C.append(num_N)
        count_payment_state_D.append(num_1)
        count_payment_state_E.append(num_3)
        
    df['count_payment_state_A_'+strs] = count_payment_state_A
    df['count_payment_state_B_'+strs] = count_payment_state_B
    df['count_payment_state_C_'+strs] = count_payment_state_C
    df['count_payment_state_D_'+strs] = count_payment_state_D
    df['count_payment_state_E_'+strs] = count_payment_state_E
    df.drop(['payment_state'],axis=1,inplace=True)
    
    return df

########################################################
### 函数功能：对辅助表中，某一列多分类指标，的每一   ###
###           个可能取值进行count，然后把对该可能取  ###
###           值的count作为一个变量。一共有k个取值， ###
###           则会引入k个变量                        ###
### 参数解释：                                       ###
###     df:      需要进行操作的 pandas               ###
###     feature: 需要进行编码的列                    ###
###     categories:   该列需要编码的，所有可能取值   ###
def count_each_cat(df, feature, categories, strs):
    result_to_merge = pd.DataFrame(df['report_id'].unique()).rename_axis({0:'report_id'},axis="columns")
    for each_categories in categories:
        col = df[df[feature] == each_categories][['report_id', feature]].groupby('report_id').count().rename_axis({feature:'count_'+each_categories+'_'+strs},axis="columns")
        col['report_id'] = col.index
        nan_dict = {'count_'+each_categories:0}
        result_to_merge = pd.merge(result_to_merge,col,how='left',on='report_id').fillna(value = nan_dict)
    return result_to_merge

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

D:\Work\Code\AboutPython\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
base_path = 'D:/DataSet/Credit/'
train = pd.read_csv(base_path + 'contest_basic_train.tsv',delimiter='\t')
test = pd.read_csv(base_path + 'contest_basic_test.tsv',delimiter='\t')
train = train[list(train.columns[-1:]) + list(train.columns[:-1])]                                # y 放到 report_id 前面
#train = train[[train.columns[0]] + list(train.columns[-1:])  + list(train.columns[1:-1])]        # y 放到 report_id 后面

hd_report = pd.read_csv(base_path + 'contest_ext_crd_hd_report.csv')                              # 报告主表
cd_ln = pd.read_csv(base_path + 'contest_ext_crd_cd_ln.tsv',delimiter='\t')                       # 贷款
cd_lnd = pd.read_csv(base_path + 'contest_ext_crd_cd_lnd.tsv',delimiter='\t')                     # 贷记卡
is_creditcue = pd.read_csv(base_path + 'contest_ext_crd_is_creditcue.csv')                        # 信用提示
is_sharedebt = pd.read_csv(base_path + 'contest_ext_crd_is_sharedebt.csv')                        # 未销户贷记卡或者未结清贷款
is_ovdsummary = pd.read_csv(base_path + 'contest_ext_crd_is_ovdsummary.csv')                      # 逾期(透支)信息汇总
qr_recordsmr = pd.read_csv(base_path + 'contest_ext_crd_qr_recordsmr.tsv',delimiter='\t')         # 查询记录汇总
qr_recorddtlinfo = pd.read_csv(base_path + 'contest_ext_crd_qr_recorddtlinfo.tsv',delimiter='\t') # 信贷审批查询记录明细
cd_ln_spl = pd.read_csv(base_path + 'contest_ext_crd_cd_ln_spl.tsv',delimiter='\t')               # 贷款特殊交易
cd_lnd_ovd = pd.read_csv(base_path + 'contest_ext_crd_cd_lnd_ovd.csv')                            # 贷记卡逾期/透支记录


def list_lower(col):
    for i in range(len(col)):
        col[i] = col[i].lower() 
    return col
columns_lower = 1
if columns_lower:
    train.columns = list_lower(list(train.columns))
    test.columns = list_lower(list(test.columns))
    hd_report.columns = list_lower(list(hd_report.columns))
    is_creditcue.columns = list_lower(list(is_creditcue.columns))
    is_sharedebt.columns = list_lower(list(is_sharedebt.columns))
    is_ovdsummary.columns = list_lower(list(is_ovdsummary.columns))
    cd_lnd_ovd.columns = list_lower(list(cd_lnd_ovd.columns))

def combine_edu(df):
    df.edu_level = df.edu_level.fillna('其他')
    ### 合并‘中学’，检验统计量：
    # 初中-高中：    0.84
    df.loc[df.edu_level == '初中','edu_level'] = '中学'
    df.loc[df.edu_level == '高中','edu_level'] = '中学'
    ### 合并‘本科及以上’，检验统计量：
    # 本-硕：        1.63
    # 本-硕及以上： -0.25
    # 本-博：        0.95
    # 硕-硕及以上： -1.24
    # 硕-博：        0.00
    # 硕及以上-博：  0.93
    df.loc[df.edu_level == '本科','edu_level'] = '本科及以上'
    df.loc[df.edu_level == '硕士研究生','edu_level'] = '本科及以上'
    df.loc[df.edu_level == '硕士及以上','edu_level'] = '本科及以上'
    df.loc[df.edu_level == '博士研究生','edu_level'] = '本科及以上'
    return df
train = combine_edu(train)
test = combine_edu(test)

def combine_marry(df):
    df.loc[df.marry_status == '离异','marry_status'] = '已婚'
    #df.loc[df.marry_status == '离婚','marry_status'] = '离婚'
    df.loc[df.marry_status == '丧偶','marry_status'] = '已婚'
    return df
train = combine_marry(train)
test = combine_marry(test)

#p1 = train.y[train.marry_status == '其他'].mean()
#p2 = train.y[train.marry_status == '离婚'].mean()
#n1 = train.y[train.marry_status == '其他'].count()
#n2 = train.y[train.marry_status == '离婚'].count()
#z(p1,p2,n1,n2)

In [3]:
# train/test 数据预处理
# 临时 function
def set_missing(df):
    df = df.drop(['id_card','loan_date'],axis=1)                          # drop 掉一些特征
    df.edu_level = df.edu_level.fillna('其他')                            # edu 缺失值填补为 其他
    df.work_province = df.work_province.fillna(df.work_province.mean())   # 这个 缺失值填补为均值
    df.salary = df.salary.fillna(df.salary.median())                      # 工资 缺失值填补为中位数
    
    df.is_local = df.is_local.fillna('本地籍')
    df.loc[df.is_local == '本地籍','is_local'] = 1
    df.loc[df.is_local == '非本地籍','is_local'] = 0
    df['is_local'] = df['is_local'].astype(np.int32)
    
    df.loc[df.edu_level == '专科', 'edu_level'] = 'junior'
    df.loc[df.edu_level == '中学', 'edu_level'] = 'middle'
    df.loc[df.edu_level == '本科及以上', 'edu_level'] = 'bachelor'
    df.loc[df.edu_level == '专科及以下', 'edu_level'] = 'under'
    df.loc[df.edu_level == '其他', 'edu_level'] = 'other'
    
    df.marry_status = df.marry_status.fillna('已婚')
    df.loc[df.marry_status == '已婚', 'marry_status'] = 'married'
    df.loc[df.marry_status == '未婚', 'marry_status'] = 'unmarried'
    df.loc[df.marry_status == '离婚', 'marry_status'] = 'divorced'
    df.loc[df.marry_status == '其他', 'marry_status'] = 'other'
    
    #df['work_pro_salary'] = df['salary'] * df['work_province']
    
    df.has_fund = df.has_fund.fillna(0)                                   # fund 缺失值填补为 0
    df.has_fund = df.has_fund.astype(np.int64)
    
    df.loc[df.agent.isnull(),'agent'] = 0
    df.loc[df.agent != 0,'agent'] = 1
    df.agent = df.agent.astype(np.int64)
    return df

train = set_missing(train)
test = set_missing(test)

# 改变 train/test 的数据类型以节省内存  
def save_memory(df):
    for each in df.columns:
        if df[each].dtypes == 'int64':
            df[each] = df[each].astype(np.int32)
        if df[each].dtypes == 'float64':
            df[each] = df[each].astype(np.float32)
    return df
train = save_memory(train)
test = save_memory(test)


In [4]:
#####################################################
############### 信用提示 is_creditcue ###############
#####################################################

# 信用提示信息
creditcue = is_creditcue.drop(['first_loan_open_month','first_loancard_open_month','first_sl_open_month','announce_count','dissent_count'],axis=1)
creditcue.columns = ['report_id', 'count_house_loan', 'count_commercial_loan','count_other_loan','count_loancard','count_standard_loancard']

# Merge
nan_dict = {'count_house_loan':0, 'count_commercial_loan':0,'count_other_loan':0,'count_loancard':0,'count_standard_loancard':0}
train = pd.merge(train,creditcue,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,creditcue,how='left',on='report_id').fillna(value = nan_dict)

# 调整特征的数据类型（均是 count 类型，全部设置为 int32）
train = set_type(train, creditcue.columns, np.int32)
test = set_type(test, creditcue.columns, np.int32)

In [5]:
########################################
############### 贷款 cd_ln #############
########################################
# 设置还款期数的天数（不定期、一次性、其他 取 1）
cd_ln['payment_rating_days'] = 1
cd_ln.loc[cd_ln.payment_rating == '按日归还','payment_rating_days'] = 1
cd_ln.loc[cd_ln.payment_rating == '按周归还','payment_rating_days'] = 7
cd_ln.loc[cd_ln.payment_rating == '按月归还','payment_rating_days'] = 30
cd_ln.loc[cd_ln.payment_rating == '按季归还','payment_rating_days'] = 91
cd_ln.loc[cd_ln.payment_rating == '按半年归还','payment_rating_days'] = 183
cd_ln.loc[cd_ln.payment_rating == '按年归还','payment_rating_days'] = 365
# 调整 剩余还款期数 和 当前逾期期数
cd_ln['remain_payment_cyc_days'] = cd_ln.remain_payment_cyc * cd_ln.payment_rating_days
cd_ln['curr_overdue_cyc_days'] = cd_ln.curr_overdue_cyc * cd_ln.payment_rating_days

# 正常账户个数 ind_normal_counts
ind_normal_counts = pd.DataFrame(cd_ln[cd_ln.state=='正常']['state'].groupby(cd_ln['report_id']).count()) 
ind_normal_counts.columns = ['ind_normal_counts']
ind_normal_counts['report_id'] = ind_normal_counts.index
train = pd.merge(train,ind_normal_counts,how='left',on='report_id').fillna(value={'ind_normal_counts':0})
test = pd.merge(test,ind_normal_counts,how='left',on='report_id').fillna(value={'ind_normal_counts':0})

# 结清账户个数 ind_clear_counts
ind_clear_counts = pd.DataFrame(cd_ln[cd_ln.state=='结清']['state'].groupby(cd_ln['report_id']).count()) 
ind_clear_counts.columns = ['ind_clear_counts']
ind_clear_counts['report_id'] = ind_clear_counts.index
train = pd.merge(train,ind_clear_counts,how='left',on='report_id').fillna(value={'ind_clear_counts':0})
test = pd.merge(test,ind_clear_counts,how='left',on='report_id').fillna(value={'ind_clear_counts':0})

# 其他账户个数 ind_other_counts
ind_other_counts = pd.DataFrame(cd_ln[cd_ln.state!='结清'][cd_ln.state!='正常']['state'].groupby(cd_ln['report_id']).count()) 
ind_other_counts.columns = ['ind_other_counts']
ind_other_counts['report_id'] = ind_other_counts.index
train = pd.merge(train,ind_other_counts,how='left',on='report_id').fillna(value={'ind_other_counts':0})
test = pd.merge(test,ind_other_counts,how='left',on='report_id').fillna(value={'ind_other_counts':0})

# 贷款机构种类数 ind_org_counts
ind_org_counts = cd_ln[['report_id','finance_org']].groupby('report_id').count()
ind_org_counts.columns = ['ind_org_counts']
ind_org_counts['report_id'] = ind_org_counts.index
nan_dict = {'ind_org_counts':0}
train = pd.merge(train,ind_org_counts,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,ind_org_counts,how='left',on='report_id').fillna(value = nan_dict)

# 每种贷款种类细分的数量
# ps.可以修改 count_each_cat 函数的第三个参数，来选择将哪些取值提取指标
train = pd.merge(train,count_each_cat(cd_ln,'type_dw',cd_ln.type_dw.unique(),'ln'),how='left',on='report_id').fillna(0)
test = pd.merge(test,count_each_cat(cd_ln,'type_dw',cd_ln.type_dw.unique(),'ln'),how='left',on='report_id').fillna(0)
#
# 每种担保方式的数量
train = pd.merge(train,count_each_cat(cd_ln,'guarantee_type',cd_ln.guarantee_type.unique(),'ln'),how='left',on='report_id').fillna(0)
test = pd.merge(test,count_each_cat(cd_ln,'guarantee_type',cd_ln.guarantee_type.unique(),'ln'),how='left',on='report_id').fillna(0)

# 五级分类
# 先合并成三级分类
cd_ln.class5_state.fillna('未知',inplace = True)
cd_ln.loc[cd_ln.class5_state == '次级','class5_state'] = '关注'
cd_ln.loc[cd_ln.class5_state == '可疑','class5_state'] = '关注'
train = pd.merge(train,count_each_cat(cd_ln,'class5_state',cd_ln.class5_state.unique(),'ln'),how='left',on='report_id').fillna(0)
test = pd.merge(test,count_each_cat(cd_ln,'class5_state',cd_ln.class5_state.unique(),'ln'),how='left',on='report_id').fillna(0)


# 24个月还款状态（分别统计 /、*、N、#、1）的总出现次数
sum_payment_state = count_payment_state(cd_ln, 'ln').groupby('report_id').sum()
sum_payment_state['report_id'] = sum_payment_state.index
train = pd.merge(train,sum_payment_state,how='left',on='report_id').fillna(0)
test = pd.merge(test,sum_payment_state,how='left',on='report_id').fillna(0)

# 合同金额、本金余额、剩余还款天数、本月应还款、本月实还款、当前逾期期天数、当前逾期金额
flt_cd_ln = cd_ln[['report_id','credit_limit_amount','balance','remain_payment_cyc_days','scheduled_payment_amount','actual_payment_amount','curr_overdue_cyc_days','curr_overdue_amount']]
flt_cd_ln = flt_cd_ln.groupby('report_id').sum()
flt_cd_ln['report_id'] = flt_cd_ln.index
train = pd.merge(train,flt_cd_ln,how='left',on='report_id').fillna(0)
test = pd.merge(test,flt_cd_ln,how='left',on='report_id').fillna(0)

# 调整数据类型
int_col = ['ind_normal_counts','ind_clear_counts','ind_other_counts','ind_org_counts','count_个人消费贷款_ln', 'count_个人经营性贷款_ln', 
           'count_其他贷款_ln','count_个人汽车贷款_ln', 'count_个人助学贷款_ln', 'count_个人住房贷款_ln', 'count_农户贷款_ln','count_个人住房公积金贷款_ln', 
           'count_个人商用房（包括商住两用）贷款_ln','count_信用/免担保_ln', 'count_组合（不含保证）担保_ln', 'count_抵押担保_ln','count_保证_ln', 
           'count_其他担保_ln', 'count_组合（含保证）担保_ln', 'count_农户联保_ln','count_质押（含保证金）担保_ln','count_正常_ln', 'count_未知_ln', 'count_关注_ln',
           'count_payment_state_A_ln', 'count_payment_state_B_ln','count_payment_state_C_ln', 'count_payment_state_D_ln','count_payment_state_E_ln',
           'curr_overdue_cyc_days']
flt_col = ['credit_limit_amount', 'balance', 'remain_payment_cyc_days','scheduled_payment_amount', 'actual_payment_amount','curr_overdue_amount']
train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

# 汉字特征重命名
rename_dict = {'count_个人消费贷款_ln':'count_consumption loan_ln',
               'count_个人经营性贷款_ln':'count_operating_loan_ln',
               'count_其他贷款_ln':'count_other_loan_ln',
               'count_个人汽车贷款_ln':'count_car_loan_ln',
               'count_个人助学贷款_ln':'count_study_loan_ln',
               'count_个人住房贷款_ln':'count_house_loan_ln',
               'count_农户贷款_ln':'count_farmer_loan_ln',
               'count_个人住房公积金贷款_ln':'count_housing_accumulation_ln',
               'count_个人商用房（包括商住两用）贷款_ln':'count_commercial_housing_ln',
               'count_信用/免担保_ln':'count_credit_loan_ln',
               'count_组合（不含保证）担保_ln':'count_combination_ln',
               'count_抵押担保_ln':'count_pledge_guarantee_ln',
               'count_保证_ln':'count_ensure_ln',
               'count_其他担保_ln':'count_other_guarantee_ln',
               'count_组合（含保证）担保_ln':'count_combination_ensure_ln',
               'count_农户联保_ln':'count_farmer_joint_ln',
               'count_质押（含保证金）担保_ln':'count_pledge_guarantee_bail_ln',
               'count_正常_ln':'count_normal_ln',
               'count_未知_ln':'count_unknown_ln',
               'count_关注_ln':'count_attention_ln',}
train = train.rename(index=str,columns=rename_dict)
test = test.rename(index=str,columns=rename_dict)

In [6]:
###########################################
############### 贷记卡 cd_lnd #############
###########################################

# 调整汇率
cd_lnd.loc[cd_lnd.currency == '人民币','currency'] = 1
cd_lnd.loc[cd_lnd.currency == '美元','currency'] = 6.5
cd_lnd.loc[cd_lnd.currency == '欧元','currency'] = 7.77
cd_lnd.loc[cd_lnd.currency == '日元','currency'] = 0.0574
cd_lnd.loc[cd_lnd.currency == '港元','currency'] = 0.83
cd_lnd.loc[cd_lnd.currency == '英镑','currency'] = 8.82
cd_lnd.loc[cd_lnd.currency == '澳大利亚元','currency'] = 5.1
cd_lnd.loc[cd_lnd.currency == '加拿大元','currency'] = 5.23
cd_lnd.loc[cd_lnd.currency == '瑞士法郎','currency'] = 6.65
cd_lnd.loc[cd_lnd.currency == '澳门元','currency'] = 0.81
set_type(cd_lnd,'currency',np.float32)

# 按照汇率，调整金额
cd_lnd['credit_limit_amount'] = cd_lnd.credit_limit_amount * cd_lnd.currency
cd_lnd['share_credit_limit_amount'] = cd_lnd.share_credit_limit_amount * cd_lnd.currency
cd_lnd['used_credit_limit_amount'] = cd_lnd.used_credit_limit_amount * cd_lnd.currency
cd_lnd['latest6_month_used_avg_amount'] = cd_lnd.latest6_month_used_avg_amount * cd_lnd.currency
cd_lnd['used_highest_amount'] = cd_lnd.used_highest_amount * cd_lnd.currency
cd_lnd['scheduled_payment_amount'] = cd_lnd.scheduled_payment_amount * cd_lnd.currency
cd_lnd['actual_payment_amount'] = cd_lnd.actual_payment_amount * cd_lnd.currency
cd_lnd['curr_overdue_amount'] = cd_lnd.curr_overdue_amount * cd_lnd.currency


# 正常账户个数 ind_normal_counts_lnd
ind_normal_counts_lnd = pd.DataFrame(cd_lnd[cd_lnd.state=='正常']['state'].groupby(cd_ln['report_id']).count()) 
ind_normal_counts_lnd.columns = ['ind_normal_counts_lnd']
ind_normal_counts_lnd['report_id'] = ind_normal_counts_lnd.index
train = pd.merge(train,ind_normal_counts_lnd,how='left',on='report_id').fillna(value={'ind_normal_counts_lnd':0})
test = pd.merge(test,ind_normal_counts_lnd,how='left',on='report_id').fillna(value={'ind_normal_counts_lnd':0})

# 结清账户个数 ind_clear_counts_lnd
ind_clear_counts_lnd = pd.DataFrame(cd_lnd[cd_lnd.state=='销户']['state'].groupby(cd_ln['report_id']).count()) 
ind_clear_counts_lnd.columns = ['ind_clear_counts_lnd']
ind_clear_counts_lnd['report_id'] = ind_clear_counts_lnd.index
train = pd.merge(train,ind_clear_counts_lnd,how='left',on='report_id').fillna(value={'ind_clear_counts_lnd':0})
test = pd.merge(test,ind_clear_counts_lnd,how='left',on='report_id').fillna(value={'ind_clear_counts_lnd':0})

# 结清账户个数 ind_unact_counts_lnd
ind_unact_counts_lnd = pd.DataFrame(cd_lnd[cd_lnd.state=='未激活']['state'].groupby(cd_ln['report_id']).count()) 
ind_unact_counts_lnd.columns = ['ind_unact_counts_lnd']
ind_unact_counts_lnd['report_id'] = ind_unact_counts_lnd.index
train = pd.merge(train,ind_unact_counts_lnd,how='left',on='report_id').fillna(value={'ind_unact_counts_lnd':0})
test = pd.merge(test,ind_unact_counts_lnd,how='left',on='report_id').fillna(value={'ind_unact_counts_lnd':0})

# 其他账户个数 ind_other_counts_lnd
ind_other_counts_lnd = pd.DataFrame(cd_lnd[cd_lnd.state!='销户'][cd_lnd.state!='正常'][cd_lnd.state!='未激活']['state'].groupby(cd_ln['report_id']).count()) 
ind_other_counts_lnd.columns = ['ind_other_counts_lnd']
ind_other_counts_lnd['report_id'] = ind_other_counts_lnd.index
train = pd.merge(train,ind_other_counts_lnd,how='left',on='report_id').fillna(value={'ind_other_counts_lnd':0})
test = pd.merge(test,ind_other_counts_lnd,how='left',on='report_id').fillna(value={'ind_other_counts_lnd':0})

# 24个月还款状态（分别统计 /、*、N、#、1）的总出现次数
sum_payment_state_lnd = count_payment_state(cd_lnd, 'lnd').groupby('report_id').sum()
sum_payment_state_lnd['report_id'] = sum_payment_state_lnd.index
train = pd.merge(train,sum_payment_state_lnd,how='left',on='report_id').fillna(0)
test = pd.merge(test,sum_payment_state_lnd,how='left',on='report_id').fillna(0)


# 贷款金额、共享额度、已用额度、最近6个月平均使用额度、最大使用额度、本月应还款、本月实还款、当前逾期金额、当前逾期期数
flt_cd_lnd = cd_lnd[['report_id','credit_limit_amount','share_credit_limit_amount','used_credit_limit_amount',
                     'latest6_month_used_avg_amount','used_highest_amount','scheduled_payment_amount',
                     'actual_payment_amount','curr_overdue_amount','curr_overdue_cyc']]
flt_cd_lnd.columns = ['report_id','credit_limit_amount_lnd','share_credit_limit_amount_lnd','used_credit_limit_amount_lnd',
                      'latest6_month_used_avg_amount_lnd','used_highest_amount_lnd','scheduled_payment_amount_lnd',
                      'actual_payment_amount_lnd','curr_overdue_amount_lnd','ind_curr_overdue_cyc_lnd']
flt_cd_lnd = flt_cd_lnd.groupby('report_id').sum()
flt_cd_lnd['report_id'] = flt_cd_lnd.index
train = pd.merge(train,flt_cd_lnd,how='left',on='report_id').fillna(0)
test = pd.merge(test,flt_cd_lnd,how='left',on='report_id').fillna(0)


# 每种担保方式的数量
train = pd.merge(train,count_each_cat(cd_lnd,'guarantee_type',cd_ln.guarantee_type.unique(),'lnd'),how='left',on='report_id').fillna(0)
test = pd.merge(test,count_each_cat(cd_lnd,'guarantee_type',cd_ln.guarantee_type.unique(),'lnd'),how='left',on='report_id').fillna(0)

# 调整数据类型
int_col = ['ind_normal_counts_lnd','ind_clear_counts_lnd','ind_unact_counts_lnd','ind_other_counts_lnd','count_payment_state_A_lnd',
           'count_payment_state_B_lnd','count_payment_state_C_lnd','count_payment_state_D_lnd','count_payment_state_E_lnd','ind_curr_overdue_cyc_lnd',
           'count_信用/免担保_lnd','count_组合（不含保证）担保_lnd','count_抵押担保_lnd','count_保证_lnd','count_其他担保_lnd',
           'count_组合（含保证）担保_lnd','count_农户联保_lnd','count_质押（含保证金）担保_lnd',]
flt_col = ['credit_limit_amount_lnd','share_credit_limit_amount_lnd','used_credit_limit_amount_lnd','latest6_month_used_avg_amount_lnd',
           'used_highest_amount_lnd','scheduled_payment_amount_lnd','actual_payment_amount_lnd','curr_overdue_amount_lnd',]
train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

# 汉字特征重命名
rename_dict = {'count_信用/免担保_lnd':'count_credit_loan_lnd',
               'count_组合（不含保证）担保_lnd':'count_combination_lnd',
               'count_抵押担保_lnd':'count_pledge_guarantee_lnd',
               'count_保证_lnd':'count_ensure_lnd',
               'count_其他担保_lnd':'count_other_guarantee_lnd',
               'count_组合（含保证）担保_lnd':'count_combination_ensure_lnd',
               'count_农户联保_lnd':'count_farmer_joint_lnd',
               'count_质押（含保证金）担保_lnd':'count_pledge_guarantee_bail_lnd',}
train = train.rename(index=str,columns=rename_dict)
test = test.rename(index=str,columns=rename_dict)

In [7]:
##############################################################
############### 贷记卡逾期/透支记录 cd_lnd_ovd ###############
##############################################################

# 逾期/透支 月数及金额
lnd_ovd = cd_lnd_ovd.groupby('report_id').sum()
lnd_ovd.columns = ['lnd_ovd_sum_last_months','lnd_ovd_sum_amount']
lnd_ovd['report_id'] = lnd_ovd.index
nan_dict = {'lnd_ovd_sum_last_months':0, 'lnd_ovd_sum_amount':0}
train = pd.merge(train,lnd_ovd,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,lnd_ovd,how='left',on='report_id').fillna(value = nan_dict)

# 逾期/透支 次数
count_ovd = cd_lnd_ovd[['report_id','month_dw']].groupby('report_id').count()
count_ovd.columns = ['count_ovd']
count_ovd['report_id'] = count_ovd.index
train = pd.merge(train,count_ovd,how='left',on='report_id').fillna(value = {'count_ovd':0})
test = pd.merge(test,count_ovd,how='left',on='report_id').fillna(value = {'count_ovd':0})

# 平均透支金额
ave_ovd_amount = pd.DataFrame(lnd_ovd.lnd_ovd_sum_amount/count_ovd.count_ovd)
ave_ovd_amount.columns = ['ave_ovd_amount']
ave_ovd_amount['report_id'] = ave_ovd_amount.index
train = pd.merge(train,ave_ovd_amount,how='left',on='report_id').fillna(value = {'ave_ovd_amount':0})
test = pd.merge(test,ave_ovd_amount,how='left',on='report_id').fillna(value = {'ave_ovd_amount':0})

# 透支金额极差
max_lnd_ovd = cd_lnd_ovd.groupby('report_id').max()
min_lnd_ovd = cd_lnd_ovd.groupby('report_id').min()
range_lnd_ovd = max_lnd_ovd.amount - min_lnd_ovd.amount
range_lnd_ovd = pd.DataFrame(range_lnd_ovd)
range_lnd_ovd.columns = ['range_lnd_ovd']
range_lnd_ovd['report_id'] = range_lnd_ovd.index
train = pd.merge(train,range_lnd_ovd,how='left',on='report_id').fillna(value = {'range_lnd_ovd':0})
test = pd.merge(test,range_lnd_ovd,how='left',on='report_id').fillna(value = {'range_lnd_ovd':0})

# 调整数据类型
int_col = ['lnd_ovd_sum_last_months','count_ovd']
flt_col = ['lnd_ovd_sum_amount','ave_ovd_amount','range_lnd_ovd']
train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

In [8]:
#######################################################################
############### 未销户贷记卡或者未结清贷款 is_sharedebt ###############
#######################################################################

# 拥有的未销户/结清的贷款种类数
count_sharedebt = is_sharedebt[['report_id','type_dw']].groupby('report_id').count()
count_sharedebt.columns = ['count_sharedebt']
count_sharedebt['report_id'] = count_sharedebt.index
train = pd.merge(train,count_sharedebt,how='left',on='report_id').fillna(value = {'count_sharedebt':0})
test = pd.merge(test,count_sharedebt,how='left',on='report_id').fillna(value = {'count_sharedebt':0})

# 未结清贷款信息汇总
not_clear_sharedebt = is_sharedebt[is_sharedebt.type_dw == '未结清贷款信息汇总'].drop(['type_dw','max_credit_limit_per_org','min_credit_limit_per_org','used_credit_limit'],axis=1)
not_clear_sharedebt.columns = ['report_id', 'not_clear_finance_corp_count', 'not_clear_finance_org_count', 'not_clear_account_count',
                               'not_clear_credit_limit', 'not_clear_balance', 'not_clear_latest_6m_used_avg_amount']
nan_dict = {'not_clear_finance_corp_count':0, 'not_clear_finance_org_count':0, 'not_clear_account_count':0,
            'not_clear_credit_limit':0, 'not_clear_balance':0, 'not_clear_latest_6m_used_avg_amount':0}
train = pd.merge(train,not_clear_sharedebt,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,not_clear_sharedebt,how='left',on='report_id').fillna(value = nan_dict)

# 未销户贷记卡信息汇总
not_logout_sharedebt = is_sharedebt[is_sharedebt.type_dw == '未销户贷记卡信息汇总'].drop(['type_dw','balance'],axis=1)
not_logout_sharedebt.columns = ['report_id','not_logout_finance_corp_count', 'not_logout_finance_org_count','not_logout_account_count', 
                                'not_logout_credit_limit', 'not_logout_max_credit_limit_per_org','not_logout_min_credit_limit_per_org', 
                                'not_logout_used_credit_limit','not_logout_latest_6m_used_avg_amount']
nan_dict = {'not_logout_finance_corp_count':0, 'not_logout_finance_org_count':0,'not_logout_account_count':0, 
            'not_logout_credit_limit':0, 'not_logout_max_credit_limit_per_org':0,'not_logout_min_credit_limit_per_org':0, 
            'not_logout_used_credit_limit':0,'not_logout_latest_6m_used_avg_amount':0}
train = pd.merge(train,not_logout_sharedebt,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,not_logout_sharedebt,how='left',on='report_id').fillna(value = nan_dict)


# 未销户贷记卡信息汇总
not_logout_pre_sharedebt = is_sharedebt[is_sharedebt.type_dw == '未销户贷记卡信息汇总'].drop(['type_dw','balance'],axis=1)
not_logout_pre_sharedebt.columns = ['report_id','not_logout_pre_finance_corp_count', 'not_logout_pre_finance_org_count',
                                    'not_logout_pre_account_count', 'not_logout_pre_credit_limit', 'not_logout_pre_max_credit_limit_per_org',
                                    'not_logout_pre_min_credit_limit_per_org', 'not_logout_pre_used_credit_limit',
                                    'not_logout_pre_latest_6m_used_avg_amount']
nan_dict = {'not_logout_pre_finance_corp_count':0, 'not_logout_pre_finance_org_count':0,
            'not_logout_pre_account_count':0, 'not_logout_pre_credit_limit':0, 'not_logout_pre_max_credit_limit_per_org':0,
            'not_logout_pre_min_credit_limit_per_org':0, 'not_logout_pre_used_credit_limit':0,
            'not_logout_pre_latest_6m_used_avg_amount':0}
train = pd.merge(train,not_logout_pre_sharedebt,how='left',on='report_id').fillna(value = nan_dict)
test = pd.merge(test,not_logout_pre_sharedebt,how='left',on='report_id').fillna(value = nan_dict)

# 调整数据类型
int_col = ['not_clear_finance_corp_count','not_clear_finance_org_count','not_clear_account_count',
           'not_logout_finance_corp_count','not_logout_finance_org_count','not_logout_account_count',
           'not_logout_pre_finance_corp_count','not_logout_pre_finance_org_count','not_logout_pre_account_count',
           'count_sharedebt']
flt_col = ['not_clear_credit_limit','not_clear_balance','not_clear_latest_6m_used_avg_amount',
           'not_logout_credit_limit','not_logout_max_credit_limit_per_org','not_logout_min_credit_limit_per_org',
           'not_logout_used_credit_limit','not_logout_latest_6m_used_avg_amount','not_logout_pre_credit_limit', 
           'not_logout_pre_max_credit_limit_per_org','not_logout_pre_min_credit_limit_per_org',
           'not_logout_pre_used_credit_limit','not_logout_pre_latest_6m_used_avg_amount']

train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

In [9]:
################################################################
############### 逾期(透支)信息汇总 is_ovdsummary ###############
################################################################

# 贷款逾期比数及月数求和
count_sum_ovdsummary = is_ovdsummary[['report_id','count_dw','months','highest_oa_per_mon','max_duration']].groupby('report_id').sum()
count_sum_ovdsummary.columns = ['count_sum_ovd_dw','count_sum_ovd_months','all_highest_oa_per_mon','all_max_duration']
count_sum_ovdsummary['report_id'] = count_sum_ovdsummary.index
nan_dict = {'count_sum_ovd_dw':0,'count_sum_ovd_months':0,'all_highest_oa_per_mon':0,'all_max_duration':0}
train = pd.merge(train,count_sum_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)
test = pd.merge(test,count_sum_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)


# 贷款逾期
count_loan_ovdsummary = is_ovdsummary[is_ovdsummary.type_dw == '贷款逾期'].drop('type_dw',axis=1)
count_loan_ovdsummary.columns = ['report_id','count_loan_ovd_dw','count_loan_ovd_months','flt_highest_loan_oa_per_mon','ind_loan_max_duration']
nan_dict = {'count_loan_ovd_dw':0,'count_loan_ovd_months':0,'flt_highest_loan_oa_per_mon':0,'ind_loan_max_duration':0}
train = pd.merge(train,count_loan_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)
test = pd.merge(test,count_loan_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)

# 贷记卡逾期
count_debit_card_ovdsummary = is_ovdsummary[is_ovdsummary.type_dw == '贷记卡逾期'].drop('type_dw',axis=1)
count_debit_card_ovdsummary.columns = ['report_id','count_debit_card_ovd_dw','count_debit_card_ovd_months','flt_highest_debit_card_oa_per_mon','ind_debit_card_max_duration']
nan_dict = {'count_debit_card_ovd_dw':0,'count_debit_card_ovd_months':0,'flt_highest_debit_card_oa_per_mon':0,'ind_debit_card_max_duration':0}
train = pd.merge(train,count_debit_card_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)
test = pd.merge(test,count_debit_card_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)

# 准贷记卡60天以上透支
count_sixty_ovdsummary = is_ovdsummary[is_ovdsummary.type_dw == '准贷记卡60天以上透支'].drop('type_dw',axis=1)
count_sixty_ovdsummary.columns = ['report_id','count_sixty_ovd_dw','count_sixty_ovd_months','flt_highest_sixty_oa_per_mon','ind_sixty_max_duration']
nan_dict = {'count_sixty_ovd_dw':0,'count_sixty_ovd_months':0,'flt_highest_sixty_oa_per_mon':0,'ind_sixty_max_duration':0}
train = pd.merge(train,count_sixty_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)
test = pd.merge(test,count_sixty_ovdsummary,how='left',on='report_id').fillna(value=nan_dict)


# 调整数据类型
int_col = ['count_sum_ovd_dw','count_sum_ovd_months','all_max_duration',
           'count_loan_ovd_dw','count_loan_ovd_months','ind_loan_max_duration',
           'count_debit_card_ovd_dw','count_debit_card_ovd_months','ind_debit_card_max_duration',
           'count_sixty_ovd_dw','count_sixty_ovd_months','ind_sixty_max_duration']
flt_col = ['all_highest_oa_per_mon','flt_highest_loan_oa_per_mon','flt_highest_debit_card_oa_per_mon','flt_highest_sixty_oa_per_mon']

train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

In [10]:
######################################################
############### 贷款特殊交易 cd_ln_spl ###############
######################################################

# 修正变更月数
cd_ln_spl.loc[cd_ln_spl.content.str.contains('提前'),'changing_months'] = -abs(cd_ln_spl[cd_ln_spl.content.str.contains('提前')]['changing_months'])
cd_ln_spl.loc[cd_ln_spl.type_dw.str.contains('提前'),'changing_months'] = -abs(cd_ln_spl[cd_ln_spl.type_dw.str.contains('提前')]['changing_months'])
cd_ln_spl.loc[cd_ln_spl.content.str.contains('展期'),'changing_months'] = abs(cd_ln_spl[cd_ln_spl.content.str.contains('展期')]['changing_months'])
cd_ln_spl.loc[cd_ln_spl.type_dw.str.contains('展期'),'changing_months'] = abs(cd_ln_spl[cd_ln_spl.type_dw.str.contains('展期')]['changing_months'])
cd_ln_spl.loc[cd_ln_spl.content.str.contains('代还'),'changing_months'] = abs(cd_ln_spl[cd_ln_spl.content.str.contains('代还')]['changing_months'])
cd_ln_spl.loc[cd_ln_spl.type_dw.str.contains('代还'),'changing_months'] = abs(cd_ln_spl[cd_ln_spl.type_dw.str.contains('代还')]['changing_months'])


# 特殊交易总次数
count_spl = cd_ln_spl[['report_id','get_time']].groupby('report_id').count()
count_spl.columns = ['count_spl']
count_spl['report_id'] = count_spl.index
train = pd.merge(train,count_spl,how='left',on='report_id').fillna(value={'count_spl':0})
test = pd.merge(test,count_spl,how='left',on='report_id').fillna(value={'count_spl':0})

# 变更月数加总
count_month = cd_ln_spl[['report_id','changing_months']].groupby('report_id').sum()
count_month.columns = ['count_month']
count_month['report_id'] = count_month.index
train = pd.merge(train,count_month,how='left',on='report_id').fillna(value={'count_month':0})
test = pd.merge(test,count_month,how='left',on='report_id').fillna(value={'count_month':0})

# 提前金额加总
cd_ln_spl['earlier_amount'] = -cd_ln_spl.changing_amount*np.sign(cd_ln_spl.changing_months)
cd_ln_spl.loc[cd_ln_spl.changing_months==0,'earlier_amount'] = cd_ln_spl.changing_amount
flt_sum_amount = cd_ln_spl[['report_id','earlier_amount']].groupby('report_id').sum()
flt_sum_amount.columns = ['flt_sum_amount']
flt_sum_amount['report_id'] = flt_sum_amount.index
train = pd.merge(train,flt_sum_amount,how='left',on='report_id').fillna(value={'flt_sum_amount':0})
test = pd.merge(test,flt_sum_amount,how='left',on='report_id').fillna(value={'flt_sum_amount':0})

# 调整数据类型
int_col = ['count_spl','count_month']
flt_col = ['flt_sum_amount']

train = set_type(train, int_col, np.int32)
train = set_type(train, flt_col, np.float32)
test = set_type(test, int_col, np.int32)
test = set_type(test, flt_col, np.float32)

In [11]:
##################################################
############### 报告主表 hd_report ###############
##################################################
# 简单 merge 即可
hd_report = hd_report.drop(['report_create_time','query_org'],axis=1)
hd_report.query_reason[hd_report.query_reason == '担保资格审查'] = 'sqe'
hd_report.query_reason[hd_report.query_reason == '贷款审批'] = 'la'
hd_report.query_reason[hd_report.query_reason == '贷后管理'] = 'mal'

hd_report.columns = ['report_id', 'cat_query_reason']
train = pd.merge(train,hd_report,how='left',on='report_id')
test = pd.merge(test,hd_report,how='left',on='report_id')

# warning!!!
# train 与 test 在 cat_query_org 上的取值不完全一样，不能直接 one-hot
# train 种只 取 “C”、“D”两个值，但是 test 另外分别有一个样本取“A”、“B”
# 经过假设检验，该指标对于结果影响较大，不可舍弃，故将去“A”、“B”的 test 样本修改为优势类：“C”
# 但是由于 get_dummies 函数仍然会分理处 “A”与“B”的取值，因此这个问题在 get_dummies 之后 drop 掉相应指标来处理
# test.loc[test.cat_query_org == 'A','cat_query_org'] = 'C'
# test.loc[test.cat_query_org == 'B','cat_query_org'] = 'C'

# 数据类型全是 obj 类型，不需要进行转换

In [12]:
#####################################################################
############### 信贷审批查询记录明细 qr_recorddtlinfo ###############
#####################################################################

# 不同查询原因的查询次数(信用卡审批次数)
ind_query_reason_0 = qr_recorddtlinfo[qr_recorddtlinfo.query_reason == '信用卡审批'][['report_id','query_reason']].groupby('report_id').count()
ind_query_reason_0.columns = ['ind_query_reason_0']
ind_query_reason_0['report_id'] = ind_query_reason_0.index
train = pd.merge(train,ind_query_reason_0,how='left',on='report_id').fillna(value={'ind_query_reason_0':0})
test = pd.merge(test,ind_query_reason_0,how='left',on='report_id').fillna(value={'ind_query_reason_0':0})

# 不同查询原因的查询次数(贷款审批次数)
ind_query_reason_1 = qr_recorddtlinfo[qr_recorddtlinfo.query_reason == '贷款审批'][['report_id','query_reason']].groupby('report_id').count()
ind_query_reason_1.columns = ['ind_query_reason_1']
ind_query_reason_1['report_id'] = ind_query_reason_1.index
train = pd.merge(train,ind_query_reason_1,how='left',on='report_id').fillna(value={'ind_query_reason_1':0})
test = pd.merge(test,ind_query_reason_1,how='left',on='report_id').fillna(value={'ind_query_reason_1':0})

# 不同查询原因的查询次数(担保资格审查次数)
ind_query_reason_2 = qr_recorddtlinfo[qr_recorddtlinfo.query_reason == '担保资格审查'][['report_id','query_reason']].groupby('report_id').count()
ind_query_reason_2.columns = ['ind_query_reason_2']
ind_query_reason_2['report_id'] = ind_query_reason_2.index
train = pd.merge(train,ind_query_reason_2,how='left',on='report_id').fillna(value={'ind_query_reason_2':0})
test = pd.merge(test,ind_query_reason_2,how='left',on='report_id').fillna(value={'ind_query_reason_2':0})

# 不同查询原因的查询次数(总查询次数)
ind_query_reason_sum = qr_recorddtlinfo[['report_id','query_reason']].groupby('report_id').count()
ind_query_reason_sum.columns = ['ind_query_reason_sum']
ind_query_reason_sum['report_id'] = ind_query_reason_sum.index
train = pd.merge(train,ind_query_reason_sum,how='left',on='report_id').fillna(value={'ind_query_reason_sum':0})
test = pd.merge(test,ind_query_reason_sum,how='left',on='report_id').fillna(value={'ind_query_reason_sum':0})

# 调整数据类型
int_col = ['ind_query_reason_0','ind_query_reason_1','ind_query_reason_2','ind_query_reason_sum']
train = set_type(train, int_col, np.int32)
test = set_type(test, int_col, np.int32)


In [13]:
# 对 obj 数据进行one-hot 编码
train = pd.get_dummies(train)
test = pd.get_dummies(test)
# train 数据集种 cat_query_org 没有“A”、“B”的取值，因此在 one-hot 之后将取“A”、“B”的指标（仅各一个样本）删除
#test = test.drop(['cat_query_org_A','cat_query_org_B'],axis=1)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Columns: 137 entries, y to cat_query_reason_sqe
dtypes: float32(37), int32(87), int64(1), uint8(12)
memory usage: 15.0 MB


In [15]:
train.isnull().sum().sum()

0

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 136 entries, report_id to cat_query_reason_sqe
dtypes: float32(37), int32(86), int64(1), uint8(12)
memory usage: 5.0 MB


In [17]:
test.to_csv(base_path+'featured/test_all_feature.csv', index=False, encoding='utf8')
train.to_csv(base_path+'featured/train_all_feature.csv', index=False, encoding='utf8')

In [147]:
print('train\t中包含记录条数（人次）共：%d；其中包含贷款者数量（人数）：%d' % (len(train.report_id), len(train.report_id.unique())) )
print('test\t中包含记录条数（人次）共：%d；其中包含贷款者数量（人数）：%d' % (len(test.report_id), len(test.report_id.unique())) )
print('所有数据集包含贷款者人数为：%d'%(len(train.report_id.unique())+len(test.report_id.unique())) )
print('===================================')
print('hd_report\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(hd_report.report_id), len(hd_report.report_id.unique())) )
print('cd_ln\t\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(cd_ln.report_id), len(cd_ln.report_id.unique())) )
print('cd_lnd\t\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(cd_lnd.report_id), len(cd_lnd.report_id.unique())) )
print('is_creditcue\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(is_creditcue.report_id), len(is_creditcue.report_id.unique())) )
print('is_sharedebt\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(is_sharedebt.report_id), len(is_sharedebt.report_id.unique())) )
print('is_ovdsummary\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(is_ovdsummary.report_id), len(is_ovdsummary.report_id.unique())) )
print('qr_recordsmr\t中包含记录条数（人次）共：%d；\t\t其中包含贷款者数量（人数）：%d' % (len(qr_recordsmr.report_id), len(qr_recordsmr.report_id.unique())) )
print('qr_recorddtlinfo中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(qr_recorddtlinfo.report_id), len(qr_recorddtlinfo.report_id.unique())) )
print('cd_ln_spl\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(cd_ln_spl.report_id), len(cd_ln_spl.report_id.unique())) )
print('cd_lnd_ovd\t中包含记录条数（人次）共：%d；\t其中包含贷款者数量（人数）：%d' % (len(cd_lnd_ovd.report_id), len(cd_lnd_ovd.report_id.unique())) )